# Supplementary figure for the BioPCA model on the toy model

Show we have an analytical understanding of what it does on the toy background model. Including W weights prediction, quite good. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import os, json
pj = os.path.join

In [ ]:
do_save_plots = True
# Resources
root_dir = pj("..", "..", "..")
data_folder = pj(root_dir, "results", "for_plots")
panels_folder = "panels/"
params_folder = pj(root_dir, "results", "common_params")

In [ ]:
# rcParams
with open(pj(params_folder, "olfaction_rcparams.json"), "r") as f:
    new_rcParams = json.load(f)
plt.rcParams.update(new_rcParams)

# color maps
with open(pj(params_folder, "back_colors.json"), "r") as f:
    all_back_colors = json.load(f)
back_color = all_back_colors["back_color"]
back_color_samples = all_back_colors["back_color_samples"]
back_palette = all_back_colors["back_palette"]

with open(pj(params_folder, "orn_colors.json"), "r") as f:
    orn_colors = json.load(f)
    
with open(pj(params_folder, "inhibitory_neuron_two_colors.json"), "r") as f:
    neuron_colors = np.asarray(json.load(f))
with open(pj(params_folder, "inhibitory_neuron_full_colors.json"), "r") as f:
    neuron_colors_full24 = np.asarray(json.load(f))
# Here, 32 neurons, need to make a new palette with same parameters
neuron_colors_full = np.asarray(sns.husl_palette(n_colors=32, h=0.01, s=0.9, l=0.4, as_cmap=False))

with open(pj(params_folder, "model_colors.json"), "r") as f:
    model_colors = json.load(f)
with open(pj(params_folder, "model_nice_names.json"), "r") as f:
    model_nice_names = json.load(f)

models = list(model_colors.keys())
print(models)

In [ ]:
# load data
ex = np.load(pj(data_folder, "sample_2d_simulation_biopca.npz"))
print(list(ex.keys()))

In [ ]:
def l2_norm(vecs, axis=-1):
    r""" Computes l2 norm of vectors stored along the last axis of vecs.
    Args:
        vecs can be either a single vector (1d) or an  arbitrary array of vectors,
            where the last dimension indexes elements of vectors.
        axis (int): which axis to sum along.

    Returns: array of distances of same shape as vecs
        except for the summation axis, removed.
    """
    return np.sqrt(np.sum(vecs**2, axis=axis))

# Panel A: Background process with PCA annotated?

In [ ]:
ex["true_pca_vecs"].shape

In [ ]:
back_vecs_example = ex["back_vecs"]
s_nuser_example = ex["nuser"]  # This has all time steps, for nicer histogram
sd_ss_vecs = np.asarray([
    0.5 * np.sum(back_vecs_example, axis=0), 
    np.diff(back_vecs_example, axis=0).flatten()
])
back_ser = sd_ss_vecs[0] + sd_ss_vecs[1] * s_nuser_example
back_ser.shape

In [ ]:
# Solution from https://stackoverflow.com/questions/41597177/get-aspect-ratio-of-axes
def get_aspect(ax):
    # Total figure size
    figW, figH = ax.get_figure().get_size_inches()
    # Axis size on figure
    _, _, w, h = ax.get_position().bounds
    # Ratio of display units
    disp_ratio = (figH * h) / (figW * w)
    # Ratio of data units
    # Negative over negative because of the order of subtraction
    data_ratio = np.diff(ax.get_ylim()) / np.diff(ax.get_xlim())

    return disp_ratio / data_ratio

In [ ]:
skp_plot = 50

# 3D plot of the synaptic weight vectors of the different neurons
fig, ax = plt.subplots()
ax.plot(0, 0, color="k", marker="o", ls="none", ms=2)
tslice = slice(1, back_ser.shape[0], skp_plot)
ax.plot(back_ser[tslice, 0], back_ser[tslice, 1], marker="o", ms=4.0, mec="k", mew=0.2,
        alpha=0.6, color=back_color_samples, lw=2.0, ls="none")
#ax.tick_params(pad=-3)
ax.set_xticks([])
ax.set_yticks([])
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.set_xlabel(r"${s}_1$", labelpad=4)
ax.set_ylabel(r"${s}_2$", labelpad=4)

# Annotate vectors
aprops = dict(arrowstyle="<-", color="k", lw=1)
input_vecs = back_vecs_example
lbls = [r"$\mathbf{s}_a$", r"$\mathbf{s}_b$", r"$\mathbf{s}_d$"]
for lbl, vec in zip(lbls, [*input_vecs[:, :2]] + [sd_ss_vecs[0, :2]]):
    ax.annotate("", xy=(0, 0), xytext=vec, arrowprops=aprops, xycoords="data", ha="center", va="center")
    ax.annotate(lbl, xy=(0, 0), xytext=vec*0.6*np.array([0.9, 1.0]), xycoords="data", ha="right")
ax.annotate("", xy=sd_ss_vecs[0, :2], xytext=sd_ss_vecs[1, :2]+sd_ss_vecs[0, :2], 
            arrowprops=aprops, xycoords="data", ha="center")
ax.annotate(r"$\mathbf{s}_s$", xy=sd_ss_vecs[0, :2], 
            xytext=(sd_ss_vecs[1, :2]+sd_ss_vecs[0, :2])*np.array([0.9, 0.95]), 
            xycoords="data", ha="right", va="top")
# PCA vector
pca_vec = ex["true_pca_vecs"][:, 0]
aprops2 = dict(arrowstyle="<->", color=model_colors["biopca"], lw=2)
ax.annotate("", xy=sd_ss_vecs[0, :2]*1.1-0.5*pca_vec[:2], xytext=sd_ss_vecs[0, :2]*1.1+0.5*pca_vec[:2], 
            arrowprops=aprops2, xycoords="data", ha="center")
aspect = get_aspect(ax)
rot_angle = np.atan(pca_vec[1]/pca_vec[0]*aspect) * 180.0 / np.pi
ax.annotate("First PC", color=model_colors["biopca"], 
            xy=sd_ss_vecs[0, :2]*1.12+0.05*sd_ss_vecs[1, :2], rotation=rot_angle[0],
            xycoords="data", ha="center", va="bottom")


axlabels = (ax.xaxis.label, ax.yaxis.label)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_biopca_toy_background_vectors.pdf"), 
               transparent=True,  bbox_inches="tight", bbox_extra_artists=axlabels)
    
plt.show()
plt.close()

# Panel B: $M$ series versus prediction (PCA vector)

#### Analytical predictions for one PCA neuron
$L$ matrix: 1x1, a scalar, equal to the inverse of the eigenvalue: $L = 1/L' = \frac{1}{\sigma^2 \| \mathbf{s}_s \|^2}$

$M$ matrix: 1x$n_R$, parallel to the fluctuating part of the background, $\mathbf{m} = \sigma^2 \| \mathbf{s}_s \| \mathbf{s}_s$.

$W$ matrix: $n_Rx1$, a vector also parallel to $\mathbf{x}_s$, we find $\mathbf{w} = \frac{\sigma^2 \| \mathbf{s}_s \|}{\sigma^2 \| \mathbf{s}_s \|^2 + \beta/\alpha} \mathbf{s}_s$. 

Then the instantaneous PN activity should be $\mathbf{y}(t) = \mathbf{s}(t) - \langle \mathbf{s} \rangle - WLM(\mathbf{s}(t) - \langle \mathbf{s} \rangle) = \frac{\beta/\alpha}{\beta/\alpha + \sigma^2 \| \mathbf{s}_s \|^2} \nu(t) \mathbf{s}_s$

In [ ]:
mser_example = ex["mser"]
tser_example = ex["tser"]
analytical_m = ex["m_pred"][0] # first PCA vector times sigma^2 times norm of s_B
dt_u = 10.0  # ms
n_neu = mser_example.shape[1]
n_orn = mser_example.shape[2]
mser_example.shape

In [ ]:
t_axis = tser_example*dt_u/1000/60
plot_skp = 20
tslice = slice(0, len(t_axis), plot_skp)
fig, ax = plt.subplots()
# Make a palette for each neuron
neuron_comp_palettes = [sns.light_palette(a, n_colors=n_orn) for a in neuron_colors[::-1]]
#neuron_comp_palettes = [sns.light_palette(model_colors["biopca"], n_colors=n_orn)]
#analytical_colors = [a[len(a)//2] for a in neuron_comp_palettes]
analytical_colors = [sns.light_palette(model_colors["biopca"], n_colors=n_orn)]

for i in range(n_orn):
    li, = ax.plot(t_axis[tslice], mser_example[tslice, 0, i], zorder=analytical_m.size + i,
                  color=neuron_comp_palettes[0][i])
    lbl = "Analytical" if i == n_orn-1 else ""
    ax.axhline(analytical_m[i], color=analytical_colors[0][i], ls="--", lw=0.65, label=lbl)
ax.set_xlabel("Time (min)")
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(3.0))
ax.legend(frameon=False)
ax.set_ylabel(r"Projection weights $m_i$")
ax.set_ylim(analytical_m.min()*1.5, analytical_m.max()*1.5)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_biopca_sample_toy_odor_m_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel C: $L$ series versus prediction (eigenvalue)

In [ ]:
lser_example = ex["lser"]
tser_example = ex["tser"]
analytical_l = ex["l_pred"][0]  # 1/sigma^2*norm(s_B^2)
lser_example.shape

In [ ]:
fig, ax = plt.subplots()
plot_skp = 40
tslice = slice(0, len(t_axis), plot_skp)
lbl = "Analytical"
analytical_colors = [model_colors["biopca"]]
# Really plot L, not L'. The BioPCA model time series is L', so take the inverse here. 
ax.axhline(1.0/analytical_l, color=analytical_colors[0], ls="--", lw=0.65, label=lbl)
li, = ax.plot(t_axis[tslice], 1.0/lser_example[tslice, 0, 0], color=neuron_comp_palettes[0][-1])

ax.set_xlabel("Time (min)")
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(3.0))
ax.legend(frameon=False)
ax.set_ylabel(r"Self-coupling $L_{11}$")
#ax.set_yscale("log")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_biopca_sample_toy_odor_l_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel D: $W$ series versus prediction

In [ ]:
wser_example = ex["wser"]
analytical_w = ex["w_pred"][0]

In [ ]:
t_axis = tser_example*dt_u/1000/60
plot_skp = 40
tslice = slice(0, len(t_axis), plot_skp)
fig, ax = plt.subplots()
# Make a palette for each neuron
neuron_comp_palettes = [sns.light_palette(a, n_colors=n_orn) for a in neuron_colors[::-1]]
#neuron_comp_palettes = [sns.light_palette(model_colors["biopca"], n_colors=n_orn)]
#analytical_colors = [a[len(a)//2] for a in neuron_comp_palettes]
analytical_colors = [sns.light_palette(model_colors["biopca"], n_colors=n_orn)]

for i in range(n_orn):
    li, = ax.plot(t_axis[tslice], wser_example[tslice, i, 0], zorder=analytical_m.size + i,
                  color=neuron_comp_palettes[0][i])
    lbl = "Analytical" if i == n_orn-1 else ""
    ax.axhline(analytical_w[i], color=analytical_colors[0][i], ls="--", lw=0.65, label=lbl)
ax.set_xlabel("Time (min)")
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(3.0))
ax.legend(frameon=False)
ax.set_ylabel(r"Inhibitory weights $w_i$")
ax.set_ylim(analytical_w.min()*1.2, analytical_w.max()*1.4)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_biopca_sample_toy_odor_w_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel E: alignment error
Just the dot product of $\mathbf{m}$ with the background $\mathbf{x}_s$ will show the alignment. Can also do $\mathbf{w}$, should be the same. 

In [ ]:
m_norm_ser = l2_norm(mser_example[:, 0], axis=1)
w_norm_ser = l2_norm(wser_example[:, :, 0], axis=1) + 1e-12
m_dot_ss = mser_example[:, 0].dot(sd_ss_vecs[1]) / m_norm_ser / l2_norm(sd_ss_vecs[1])
w_dot_ss = wser_example[:, :, 0].dot(sd_ss_vecs[1]) / w_norm_ser / l2_norm(sd_ss_vecs[1])

In [ ]:
plot_skp = 20
tslice = slice(0, len(t_axis), plot_skp)

fig, ax = plt.subplots()
lim, = ax.plot(t_axis[tslice], 1.0 - m_dot_ss[tslice], ls="-", color=neuron_comp_palettes[0][-1], label=r"$\mathbf{m}$")
liw, = ax.plot(t_axis[tslice], 1.0 - w_dot_ss[tslice], ls="--", color=neuron_comp_palettes[0][-1], label=r"$\mathbf{w}$")

ax.set_xlabel("Time (min)")
ax.xaxis.set_major_locator(mpl.ticker.MultipleLocator(3.0))
ax.legend(frameon=False)
ax.set_ylabel(r"Alignment error, $1 - \frac{\mathbf{x} \cdot \mathbf{s}_s}{\|\mathbf{x}\| \|\mathbf{s}_s\|}$")
ax.set_yscale("log")
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_biopca_sample_toy_odor_m_align_series.pdf"), 
                transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Panel F: background inhibition versus prediction

In [ ]:
yser = ex["yser"]
yser_norm = l2_norm(yser, axis=1)
yser_pred = ex["yser_pred"]

yser_norm_pred = l2_norm(yser_pred, axis=1)
yser_norm_pred_avg = np.mean(yser_norm_pred[-400:])
yvari_pred = ex["yvari_pred"][0]

nuser_example = ex["nuser"]
sser_example = (0.5+nuser_example)*ex["back_vecs"][0] + (0.5 - nuser_example)*ex["back_vecs"][1]
sser_norm = l2_norm(sser_example)

In [ ]:
plot_skp = 40
tslice = slice(1, len(t_axis), plot_skp)

fig, ax = plt.subplots()
ax.plot(t_axis[tslice], sser_norm[tslice], color=back_color, label=r"Background $\|\mathbf{s}\|$", lw=0.75)
ax.plot(t_axis[tslice], yser_norm[tslice], color=neuron_comp_palettes[0][-1], label=r"BioPCA sim. $\|\mathbf{y}\|$", lw=0.75)
ax.plot(t_axis[tslice], yser_norm_pred[tslice], label=r"BioPCA pred. $\|\mathbf{y}\|$", lw=0.5, ls=":",
       color=(mpl.colors.to_rgba(model_colors["biopca"]) + np.asarray([0.25,]*3 + [0])).clip(0, 1))
ax.set(xlabel="Time (min)", ylabel=r"PN activity norm, $\|\mathbf{y}\|$", yscale="log")
ax.set_xticks(np.arange(0.0, t_axis[-1], 2.0))
ax.axhline(yser_norm_pred_avg, ls="-.", label=r"Average pred. $\langle \|\mathbf{y}\| \rangle$",
           color=(mpl.colors.to_rgba(model_colors["biopca"]) - np.asarray([0.25,]*3 + [0])).clip(0, 1))
leg = ax.legend(bbox_to_anchor=(1.1, -0.03), loc="lower right", frameon=False, labelspacing=0.2)
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj(panels_folder, "supfig_biopca_sample_toy_odor_y_series.pdf"), 
                transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()